In [1]:
import torch
from torchvision import transforms
from torchvision.transforms import functional as F
import cv2
import csv
import time
import pandas as pd
import numpy as np
import os
from PIL import Image, ImageDraw
from pathlib import Path
from tqdm import tqdm
import subprocess
import torch.nn as nn
import statistics
from scipy.spatial.distance import cdist
from filterpy.kalman import KalmanFilter

In [2]:
# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
model.eval()
# class Image_Classifier(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.model = nn.Sequential(
#             nn.Conv2d(3, 32,(3,3)),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Conv2d(32, 64,(3,3)),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Conv2d(64, 128,(3,3)),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Conv2d(128, 256,(3,3)),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Conv2d(256, 256,(3,3)),
#             nn.ReLU(),
#             nn.MaxPool2d(2,2),
#             nn.Flatten(),
#             nn.Linear(9216, 10),
#         )
#         # Define the class labels
#         self.classes = ['Bus', 'Heavy-Duty', 'Lorry', 'Luxury', 'Motorbike', 'Pickup', 'SUV', 'Sedan', 'Three Wheel', 'Van']
            
#     def forward(self, x):
#         return self.model(x)
# # Load Bedo model for car classification
# classifier = Image_Classifier()
# classifier.load_state_dict(torch.load("Bedo_model.pt"))
# classifier = classifier.cuda()
classifier = torch.load('model.pt')
classifier.eval()

Using cache found in C:\Users\ahmed/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-4-18 Python-3.9.13 torch-2.0.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 6144MiB)



requirements: C:\Users\ahmed\.cache\torch\hub\requirements.txt not found, check failed.


Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [25]:
def predict_color(cropped_image):
    
    # Apply Gaussian blur to reduce noise
    blurred_image = cv2.GaussianBlur(cropped_image, (5, 5), 0)
    
    hsv_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2HSV)
    
    color_ranges = {
        'red': ([0, 70, 50], [10, 255, 255]),
        'green': ([36, 70, 50], [70, 255, 255]),
        'blue': ([90, 70, 50], [130, 255, 255]),
        'yellow': ([20, 70, 50], [35, 255, 255]),
        'orange': ([10, 70, 50], [25, 255, 255]),
        'purple': ([130, 70, 50], [160, 255, 255]),
        'grey': ([0, 0, 50], [180, 30, 220]),
        'black': ([0, 0, 0], [180, 255, 30]),
        'white': ([0, 0, 220], [180, 30, 255]),
        'pink': ([150, 70, 50], [170, 255, 255]),
        'brown': ([10, 100, 20], [20, 255, 200])}
    
    predicted_colors = ['unknown'] * 3
    max_counts = [0] * 3
    
    # Loop through the color ranges and check if the cropped image falls within any of them
    for color, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        mask = cv2.inRange(hsv_image, lower, upper)
        count = cv2.countNonZero(mask)
        
        # Update the top three colors and their counts
        for i in range(3):
            if count > max_counts[i]:
                predicted_colors.insert(i, color)
                max_counts.insert(i, count)
                predicted_colors.pop()
                max_counts.pop()
                break
    
    return predicted_colors[1]


In [23]:
def detect_Bedo(video_path, output_path):

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get the frames per second and frame size of the video
    fps = cap.get(cv2.CAP_PROP_FPS) 
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) 
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  

    # Define the ffmpeg command to write the output video  
    ffmpeg_cmd = f"ffmpeg -y -f rawvideo -pix_fmt bgr24 -s {frame_width}x{frame_height} -r {fps} -i - -c:v libx264 -preset fast -crf 30 -pix_fmt nv12 -an -vcodec libx264 {output_path}"

    # Open the output file for writing  
    output_file = subprocess.Popen(ffmpeg_cmd.split(' '), stdin=subprocess.PIPE)
    
    object_dict = {'color_classifier_preds': [], 'color_mode_pred': None}
    
    body_name = ['Heavy-Duty', 'Lorry', 'Luxury', 'Pickup', 'SUV', 'Sedan', 'Van']
    # Loop through the frames of the video
    while cap.isOpened():
        # Read the next frame from the video
        ret, frame = cap.read()

        # If there are no more frames, break out of the loop
        if not ret:
            print('complete video')
            break

        # Run the frame through the YOLOv5 model
        yolov5_results = model(frame)

        # Check that the results contain valid bounding boxes
        if len(yolov5_results.xyxy[0]) == 0:
            continue

        # Loop through the bounding boxes and classify the cars
        for label, box, conf in zip(yolov5_results.xyxy[0][:, -1], yolov5_results.xyxy[0][:, :4], yolov5_results.xyxy[0][:, 4]):
            box = [round(i, 2) for i in box.tolist()]
            class_label = model.names[int(label)]

            # Check if the class label is "car", "truck", "bus", "bicycle", or "motorcycle"
            if class_label in ["car", "truck", "bus", "bicycle", "motorcycle"]:
                # Ensure that the bounding box is within the bounds of the frame
                x1, y1, x2, y2 = box
                x1, y1, x2, y2 = max(0, int(x1)), max(0, int(y1)), min(frame.shape[1], int(x2)), min(frame.shape[0], int(y2))
                if x1 >= x2 or y1 >= y2:
                    continue
                    
                # Crop the car image from the frame
                predicted_color = predict_color(frame[y1:y2, x1:x2])
                
                object_dict['color_classifier_preds'].append(predicted_color)
                
                car_image = frame[y1:y2, x1:x2]

                # Resize the car image to 256x256 using OpenCV functions
                car_image = cv2.resize(car_image, (256, 256), interpolation=cv2.INTER_LINEAR)

                # Convert the car image to a PyTorch tensor
                car_image = torch.from_numpy(car_image).permute(2, 0, 1).float().cuda()
                car_image /= 255.0
                
                object_dict['color_mode_pred'] = statistics.mode(object_dict['color_classifier_preds'])
                color_mode_pred = object_dict['color_mode_pred']
                
                # Classify the car image using the Bedo model
                with torch.no_grad():
                    output = classifier(car_image.unsqueeze(0))
                    prediction = torch.argmax(output).item()
                    class_name = body_name[prediction]

                # Draw the bounding box and class label on the frame
                font_scale = 1
                thickness = 1  # Set the thickness to a larger value for a bolder font
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
                cv2.putText(frame, class_name, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (203, 192, 255), thickness+1)
                cv2.putText(frame, predicted_color, (x1, y2), cv2.FONT_HERSHEY_SIMPLEX, font_scale, (203, 192, 255), thickness+1)

        # Write the frame to the output video
        output_file.stdin.write(frame.tobytes())

    # Release the video file and close the output video
    cap.release() 
    output_file.stdin.close() 
    output_file.wait()

In [26]:
detect_Bedo('light traffic.mp4','test2.mp4')

complete video
